In [76]:
from typing import *
from itertools import combinations
import numpy as np


In [77]:
def min_cost_partition(nr_qubits: int,
                       k: int,
                       mu: np.ndarray,
                       sigma: np.ndarray,
                       alpha: float) -> Tuple[dict, dict, float]:
    
    def generate_binary_combinations(n: int, k: int) -> np.ndarray:
        """ Generates all the 'n' chose 'k' combinations w. 'k' ones. """
        num_permutations = 2 ** n
        for indices in combinations(range(n), k):
            # Create a numpy array of zeros of size N
            arr = np.zeros(n, dtype=int)
            # Set ones at the specified positions
            arr[list(indices)] = 1
            yield arr

    def generate_binary_permutations(n: int) -> np.ndarray:
        """ Generates all the 2^n permutations of bitstring w. length 'n'. """
        num_permutations = 2 ** n
        for i in range(num_permutations):
            _binary_string_ = bin(i)[2:].zfill(n)
            yield np.array([int(bit) for bit in _binary_string_])

    def cost(state: np.ndarray, mu: np.ndarray, sigma: np.ndarray, alpha: float) -> float:
        return -np.dot(state, mu)  + alpha*np.dot(state, np.dot(sigma, state))

    max_cost_1, min_cost_1, min_comb = -np.inf, np.inf, np.empty(shape=(nr_qubits,))
    for comb in generate_binary_combinations(n=nr_qubits, k=k):
        comb_cost = cost(state=comb, mu=mu, sigma=sigma, alpha=alpha)
        if comb_cost < min_cost_1:
            min_cost_1, min_comb = comb_cost, comb
        if comb_cost > max_cost_1:
            max_cost_1 = comb_cost
    binary_comb = min_comb

    max_cost_2, min_cost_2, min_perm = -np.inf, np.inf, np.empty(shape=(nr_qubits,))
    for perm in generate_binary_permutations(n=nr_qubits):
        perm_cost = cost(state=perm, mu=mu, sigma=sigma, alpha=alpha)
        if perm_cost < min_cost_2:
            min_cost_2, min_perm= perm_cost, perm
        if perm_cost > max_cost_2:
            max_cost_2 = perm_cost
    binary_perm = min_perm 

    lmbda = 0
    if min_cost_2 < min_cost_1:
        lmbda = abs(min_cost_2) - abs(min_cost_1)

    constrained_result = {'s': binary_comb, 'c_min': min_cost_1, 'c_max': max_cost_1}
    full_result = {'s': binary_perm, 'c_min': min_cost_2, 'c_max': max_cost_2}
    return constrained_result, full_result, lmbda


def get_Q(mu: np.ndarray, sigma: np.ndarray, alpha: float, lmbda: float, k: int) -> tuple[np.ndarray, float]:
    Q = np.zeros_like(sigma)
    N = Q.shape[0]
    for i in range(N):
        for j in range(N):
            if i == j:
                Q[i,j] += -mu[i] - 2*k*lmbda + lmbda + alpha*sigma[i,j] 
            else:
                Q[i,j] += lmbda + alpha*sigma[i,j]
    offset = lmbda*k**2
    return Q, offset
    

In [78]:
N=4
k=2
alpha = 0.001
seed = 4
np.random.seed(seed)

mu = np.random.uniform(low=0, high=0.99, size=N)

_temp_ = np.random.uniform(low=0, high=0.99, size=(N, N))
sigma = np.dot(_temp_, _temp_.transpose())
if not np.all(sigma == sigma.T) or not np.all(np.linalg.eigvals(sigma) >= 0):
    raise ValueError('Covariance matrix is not PSD.')

In [79]:
n_chose_k, full, lmbda = min_cost_partition(nr_qubits=N, k=k, mu=mu, sigma=sigma, alpha=alpha)

In [80]:
n_chose_k['s'], full['s'], lmbda

(array([1, 0, 1, 0]), array([1, 1, 1, 1]), 1.2407897066063132)

In [81]:
def generate_binary_permutations(n: int) -> np.ndarray:
    """ Generates all the 2^n permutations of bitstring w. length 'n'. """
    num_permutations = 2 ** n
    for i in range(num_permutations):
        _binary_string_ = bin(i)[2:].zfill(n)
        yield np.array([int(bit) for bit in _binary_string_])
        
def full_cost(state: np.ndarray, mu: np.ndarray, sigma: np.ndarray, alpha: float, lmbda: float, k: int) -> float:
    return -np.dot(state, mu)  + alpha*np.dot(state, np.dot(sigma, state)) + lmbda*(np.dot(state,np.ones_like(state))-k)**2

min_cost, min_perm = np.inf, None
for perm in generate_binary_permutations(n=N):
    cost = full_cost(state=perm, mu=mu, sigma=sigma, alpha=alpha, lmbda=lmbda, k=k)
    if cost < min_cost:
        min_cost = cost
        min_perm = perm

print(min_perm)

[1 0 1 0]


In [82]:
def get_Q(mu: np.ndarray, sigma: np.ndarray, alpha: float, lmbda: float, k: int) -> tuple[np.ndarray, float]:
    Q = np.zeros_like(sigma)
    N = Q.shape[0]
    for i in range(N):
        for j in range(N):
            if i == j:
                Q[i,j] += -mu[i] - 2*k*lmbda + lmbda + alpha*sigma[i,j] 
            else:
                Q[i,j] += lmbda + alpha*sigma[i,j]
    offset = lmbda*k**2
    return Q, offset

In [83]:
Q, offset = get_Q(mu=mu, sigma=sigma, alpha=alpha, lmbda=lmbda, k=k)
for perm in generate_binary_permutations(n=N):
    cost = full_cost(state=perm, mu=mu, sigma=sigma, alpha=alpha, lmbda=lmbda, k=k)
    qubo_cost = np.dot(perm, np.dot(Q, perm)) + offset
    print(cost, qubo_cost)

4.963158826425253 4.963158826425253
0.5341672995895156 0.5341672995895159
0.2798859764001691 0.27988597640016977
-1.6656314353934103 -1.6656314353934087
0.6999114395406706 0.6999114395406707
-1.246728580993889 -1.2467285809938886
-1.500034133574851 -1.5000341335748502
-0.9632000390672528 -0.9632000390672522
0.2848872520435588 0.2848872520435588
-1.6622526360966403 -1.6622526360966399
-1.9148884929800205 -1.9148884929800198
-1.3785542660780619 -1.3785542660780603
-1.4947566382501996 -1.494756638250199
-0.9595450200892215 -0.9595450200892204
-1.2112051063642169 -1.2112051063642157
1.807480626838919 1.8074806268389203


In [84]:
1.6656314353934103*2

3.3312628707868206